In [74]:
import pandas as pd
import numpy as np

from ta import momentum, trend, volume
from binance import Client
from key import PUB_KEY, SEC_KEY

# Get Historical Data

In [75]:
# Connecting to Binance
client = Client(PUB_KEY, SEC_KEY)
# Get historical data
df = pd.DataFrame(client.get_historical_klines('BTCUSDT',
                             Client.KLINE_INTERVAL_1MINUTE,
                                      str(500) + ' minutes ago UTC'))

# First 6 column represent Time and OHLCV values
df = df.iloc[:, :6]
df.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']

# Convert first column into datetime
df['Time'] = pd.to_datetime(df['Time'], unit='ms')
for column in df.columns:
    if column != 'Time':
        df[column] = df[column].astype(float)

# Set Time as index
df.set_index('Time', inplace=True)

# Generate Technical Indicators

In [77]:
df

,Open,High,Low,Close,Volume
Time,,,,,
2024-07-10 02:36:00,57730.00,57793.96,57729.99,57793.96,4.45281
2024-07-10 02:37:00,57793.96,57826.00,57793.95,57804.01,8.51498
2024-07-10 02:38:00,57804.00,57804.01,57760.00,57770.00,4.89333
2024-07-10 02:39:00,57769.99,57770.00,57720.12,57730.01,6.25034
2024-07-10 02:40:00,57730.00,57809.79,57730.00,57802.01,6.07886
...,...,...,...,...,...
2024-07-10 10:51:00,58666.23,58715.95,58620.62,58620.63,20.88917
2024-07-10 10:52:00,58620.63,58650.00,58615.07,58650.00,8.64937
2024-07-10 10:53:00,58650.00,58656.87,58649.65,58656.86,6.32817


In [78]:
df['SMA'] = df['Close'].rolling(20).mean()
df['RSI'] = momentum.rsi(df['Close'])
df['MACD'] = trend.macd_diff(df['Close'])
df['Momentum'] = (df['Close'] - df['Close'].shift(30))/df['Close'].shift(30)
df['ADX'] = trend.adx(df['High'], df['Low'], df['Close'])
df['-DI'] = trend.adx_neg(df['High'], df['Low'], df['Close'])
df['+DI'] = trend.adx_pos(df['High'], df['Low'], df['Close'])


c:\Users\nolan\AppData\Local\Programs\Python\Python310\lib\site-packages\ta\trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
c:\Users\nolan\AppData\Local\Programs\Python\Python310\lib\site-packages\ta\trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


# Generate Trading Signals

In [79]:
df.dropna(inplace=True)
df

,Open,High,Low,Close,Volume,ADX,-DI,+DI,SMA,RSI,MACD,Momentum
Time,,,,,,,,,,,,
2024-07-10 03:09:00,57960.13,57960.13,57940.57,57946.00,6.81999,22.462481,17.985541,34.328185,57892.1835,63.683194,2.165850,0.003741
2024-07-10 03:10:00,57946.00,57950.00,57929.36,57929.37,6.13389,22.612590,19.768688,32.643128,57900.4305,59.473227,-0.519620,0.002203
2024-07-10 03:11:00,57929.36,57930.00,57885.50,57885.51,9.04040,21.181589,27.830128,29.303361,57904.8495,50.071537,-5.335508,0.000866
2024-07-10 03:12:00,57885.51,57924.00,57875.00,57924.00,5.70246,19.803331,27.137500,26.132833,57909.8495,56.561253,-5.966698,0.001183
2024-07-10 03:13:00,57923.99,57932.00,57910.00,57910.01,2.73180,18.505026,25.788383,26.641448,57912.1650,53.822842,-7.277465,0.002010
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-10 10:51:00,58666.23,58715.95,58620.62,58620.63,20.88917,31.579736,17.056770,39.304262,58646.6900,50.364304,-7.325845,0.002076
2024-07-10 10:52:00,58620.63,58650.00,58615.07,58650.00,8.64937,31.944405,16.977972,36.652279,58647.6900,54.358217,-6.390058,0.001308
2024-07-10 10:53:00,58650.00,58656.87,58649.65,58656.86,6.32817,32.400413,16.726745,37.517917,58649.1425,55.263677,-5.264906,0.001791
